# BT4012 Team 19 Group Project

## Team members
+ LAW HIU YAN (AXXXXXXX)
+ LIM JING YU (AXXXXXXX)
+ NG XIANG HAN (A0220690M)
+ SEAH JIA JUN (A0218067A)
+ TAN JIA HUI (A0223956X)

## 1. Importing the required libraries

In [21]:
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## 2. Loading Dataset

In [22]:
dataset = pd.read_csv('./transaction_dataset.csv')
dataset.head()

,Unnamed: 0,Index,Address,FLAG,Avg min between sent tnx,Avg min between received tnx,Time Diff between first and last (Mins),Sent tnx,Received Tnx,Number of Created Contracts,...,ERC20 min val sent,ERC20 max val sent,ERC20 avg val sent,ERC20 min val sent contract,ERC20 max val sent contract,ERC20 avg val sent contract,ERC20 uniq sent token name,ERC20 uniq rec token name,ERC20 most sent token type,ERC20_most_rec_token_type
0,0,1,0x00009277775ac7d0d59eaad8fee3d10ac6c805e8,0,844.26,1093.71,704785.63,721,89,0,...,0.000000,1.683100e+07,271779.920000,0.0,0.0,0.0,39.0,57.0,Cofoundit,Numeraire
1,1,2,0x0002b44ddb1476db43c868bd494422ee4c136fed,0,12709.07,2958.44,1218216.73,94,8,0,...,2.260809,2.260809e+00,2.260809,0.0,0.0,0.0,1.0,7.0,Livepeer Token,Livepeer Token
2,2,3,0x0002bda54cb772d040f779e88eb453cac0daa244,0,246194.54,2434.02,516729.30,2,10,0,...,0.000000,0.000000e+00,0.000000,0.0,0.0,0.0,0.0,8.0,NaN,XENON
3,3,4,0x00038e6ba2fd5c09aedb96697c8d7b8fa6632e5e,0,10219.60,15785.09,397555.90,25,9,0,...,100.000000,9.029231e+03,3804.076893,0.0,0.0,0.0,1.0,11.0,Raiden,XENON
4,4,5,0x00062d1dd1afb6fb02540ddad9cdebfe568e0d89,0,36.61,10707.77,382472.42,4598,20,1,...,0.000000,4.500000e+04,13726.659220,0.0,0.0,0.0,6.0,27.0,StatusNetwork,EOS


In [23]:
print("The shape of the dataset is: " + str(dataset.shape))

The shape of the dataset is: (9841, 51)


In [24]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9841 entries, 0 to 9840
Data columns (total 51 columns):
 #   Column                                                Non-Null Count  Dtype  
---  ------                                                --------------  -----  
 0   Unnamed: 0                                            9841 non-null   int64  
 1   Index                                                 9841 non-null   int64  
 2   Address                                               9841 non-null   object 
 3   FLAG                                                  9841 non-null   int64  
 4   Avg min between sent tnx                              9841 non-null   float64
 5   Avg min between received tnx                          9841 non-null   float64
 6   Time Diff between first and last (Mins)               9841 non-null   float64
 7   Sent tnx                                              9841 non-null   int64  
 8   Received Tnx                                          9841

## 3. Data Cleaning

In [25]:
#Drop index-related columns that are not useful
dataset.drop(columns=['Unnamed: 0', "Index"], axis=1, inplace=True)

In [26]:
#Rename target variable as Y
dataset = dataset.rename(columns={'FLAG':'Y'})

In [27]:
# Count number of missing values for each column, sort them in descending order
top_missing_columns = dataset.isnull().sum().sort_values(ascending=False)
top_missing_columns

 ERC20 most sent token type                             2697
 ERC20_most_rec_token_type                               871
 ERC20 min val rec                                       829
 ERC20 total Ether sent contract                         829
 ERC20 uniq sent addr                                    829
 ERC20 uniq rec addr                                     829
 ERC20 uniq sent addr.1                                  829
 ERC20 uniq rec contract addr                            829
 ERC20 avg time between sent tnx                         829
 ERC20 avg time between rec tnx                          829
 ERC20 avg time between rec 2 tnx                        829
 ERC20 avg time between contract tnx                     829
 ERC20 max val rec                                       829
 ERC20 total Ether received                              829
 ERC20 avg val rec                                       829
 ERC20 min val sent                                      829
 ERC20 max val sent     

In [28]:
#inspect values of ERC20 most sent token type to decide how to impute values
dataset[' ERC20 most sent token type'].value_counts()

 ERC20 most sent token type
0                                                         4399
                                                          1191
EOS                                                        138
OmiseGO                                                    137
Golem                                                      130
                                                          ... 
BlockchainPoland                                             1
Covalent Token                                               1
Nebula AI Token                                              1
Blocktix                                                     1
eosDAC Community Owned EOS Block Producer ERC20 Tokens       1
Name: count, Length: 304, dtype: int64

We see that the highest count of ERC20 most sent token type is a token called '0' at a count of 4399, followed by '' at a count of 1191.
From research, these tokens do not exist. It is also unlikely that '0' and '' are actual token names, due to their lack of distinctiveness.
Categorise this column into observations with meaningful token names vs not meaningful names (such as '0', '' and missing values)

In [29]:
#mapping function to map 'ERC 20 most sent token type'
def map_token_type(token_name):
    if (token_name in ['0', "''"]):
        return 0
    else:
        return 1

#replace empty values with 0
dataset[' ERC20 most sent token type'].fillna(0, inplace=True)
#map remainder of the values
dataset[' ERC20 most sent token type'] = dataset[' ERC20 most sent token type'].apply(map_token_type)
#rename column
dataset = dataset.rename(columns={' ERC20 most sent token type': 'ERC20_most_sent_token_valid_name'})

In [30]:
#nature of column ' ERC20_most_rec_token_type' is similar to previous case, apply same transformation
#replace empty values with 0
dataset[' ERC20_most_rec_token_type'].fillna(0, inplace=True)
#map remainder of the values
dataset[' ERC20_most_rec_token_type'] = dataset[' ERC20_most_rec_token_type'].apply(map_token_type)
#rename column
dataset = dataset.rename(columns={' ERC20_most_rec_token_type': 'ERC20_most_rec_token_valid_name'})

For the remainder of the columns with missing values, we see that the count of rows qith missing values are consistently 829. It is possible that these observations do not involve ERC20 tokens since ETH can be of many kinds. Remove these columns since they will not help with the fraud detection.

In [31]:
#drop rows with null values from remaining dataset
dataset = dataset.dropna()

In [32]:
#inspect resultant df to check that there are not more missing values
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9012 entries, 0 to 9840
Data columns (total 49 columns):
 #   Column                                                Non-Null Count  Dtype  
---  ------                                                --------------  -----  
 0   Address                                               9012 non-null   object 
 1   Y                                                     9012 non-null   int64  
 2   Avg min between sent tnx                              9012 non-null   float64
 3   Avg min between received tnx                          9012 non-null   float64
 4   Time Diff between first and last (Mins)               9012 non-null   float64
 5   Sent tnx                                              9012 non-null   int64  
 6   Received Tnx                                          9012 non-null   int64  
 7   Number of Created Contracts                           9012 non-null   int64  
 8   Unique Received From Addresses                        9012 non-

There are no more missing values. Our resulting dataframe has 48 columns and 9012 observations.

In [33]:
#check for less or zero variance
numeric_columns = dataset.select_dtypes(include=['int64', 'float64'])

# Calculate the variance for each numeric column
for column in numeric_columns.columns:
    column_variance = numeric_columns[column].var()
    print(f"Variance in '{column}': {column_variance}")

Variance in 'Y': 0.12737428037724569
Variance in 'Avg min between sent tnx': 490087973.1829964
Variance in 'Avg min between received tnx': 560986033.7290806
Variance in 'Time Diff between first and last (Mins)': 109868780959.2777
Variance in 'Sent tnx': 624821.5182641346
Variance in 'Received Tnx': 963108.8348077973
Variance in 'Number of Created Contracts': 21846.176775643296
Variance in 'Unique Received From Addresses': 97294.79782001168
Variance in 'Unique Sent To Addresses': 75942.02713420542
Variance in 'min value received': 113090.44684189241
Variance in 'max value received ': 184768949.46628866
Variance in 'avg val received': 9085308.570906386
Variance in 'min val sent': 20973.979238544995
Variance in 'max val sent': 47977362.61730275
Variance in 'avg val sent': 61813.3141368055
Variance in 'min value sent to contract': 5.547665443966177e-08
Variance in 'max val sent to contract': 2.9053683593026035e-07
Variance in 'avg value sent to contract': 1.1423064430050306e-07
Variance in

From above, we see that the following variables have zero variance
* ' ERC20 avg time between sent tnx'
* ' ERC20 avg time between rec tnx'
* ' ERC20 avg time between rec 2 tnx'
* ' ERC20 avg time between contract tnx'
* ' ERC20 avg time between rec 2 tnx'
* ' ERC20 max val sent contract'
* ' ERC20 avg val sent contract'

And the following variables have extremely low variance, close to zero:
* 'min value sent to contract'
* 'max val sent to contract'
* 'avg value sent to contract'
* 'total ether sent contracts'

We can see that these variables are related to smart contracts.

In [34]:
#Drop index-related columns that are not useful
dataset.drop(columns=[' ERC20 avg time between sent tnx', ' ERC20 avg time between rec tnx', ' ERC20 avg time between rec 2 tnx', ' ERC20 avg time between contract tnx',
' ERC20 avg time between rec 2 tnx', ' ERC20 max val sent contract', ' ERC20 avg val sent contract', 'min value sent to contract', 'max val sent to contract',
'avg value sent to contract', 'total ether sent contracts'], axis=1, inplace=True)

## 4. EDA